## Toy example of 1D linear regression with Keras
To demonstrate the basic workflow

### Create features and targets ("X" and "Y")

In [ ]:
import numpy as np

hidden_slope = 2.0
hidden_bias = 5.0

hidden_model = lambda x: hidden_slope * x + hidden_bias

samples = 100

X = np.linspace(1, 10, samples)
X = np.reshape(X, (-1, 1))

Y_noise_std = 10

Y = hidden_model(X) + np.random.randn(X.shape[0], X.shape[1]) * Y_noise_std

print('X shape: (%d, %d)' % X.shape)
print('Y shape: (%d, %d)' % Y.shape)

### Visualize features and targets

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(X, Y, 'bo')
plt.plot(X, hidden_model(X), 'r--')

### Split data to training and validation sets and preprocess using StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)

Y_scaler = StandardScaler()
Y_train = Y_scaler.fit_transform(Y_train)

plt.plot(X_train, Y_train, 'bo')
plt.plot(X_scaler.transform(X), Y_scaler.transform(hidden_model(X)), 'r--')
plt.xlabel('X scaled')
plt.ylabel('Y scaled')

### Create Keras model with one layer containing one node with linear activation

In [ ]:
from keras.models import Sequential
model = Sequential()

from keras.layers import Dense
from keras.activations import linear
model.add(Dense(1, activation=linear, input_dim = 1))

from keras import optimizers
optimizer = optimizers.SGD(lr = 0.01)

from keras import losses
loss = losses.mean_squared_error

model.compile(optimizer = optimizer, loss = loss, metrics = ['mse'])

model.summary()

### Train the model, plot training and validation loss vs. epochs

In [ ]:
history = model.fit(X_train, Y_train, epochs = 50, verbose = 0, validation_split=0.10)

history_dict = history.history

plt.plot(history_dict['loss'], 'bo-', label = 'Training loss')
plt.plot(history_dict['val_loss'], 'rs--', label = 'Validation loss')

plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

### Predict targets for test set, plot and compare to model noise

In [ ]:
Y_predicted = model.predict(X_scaler.transform(X_test))

Y_predicted = Y_scaler.inverse_transform(Y_predicted)

plt.plot(X_test, Y_test, 'bo', label = 'Test set')
plt.plot(X_test, Y_predicted, 'ro', label = 'Predicted')
plt.plot(X, hidden_model(X), 'k--', label = 'Hidden model')
plt.legend()

from sklearn.metrics import mean_squared_error
print('RMS error %.1f (noise %.1f)' % (np.sqrt(mean_squared_error(Y_test, Y_predicted)), Y_noise_std))